In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/MIP_3_Thermal_Plants/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-22 21:02:32.709632


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- $i$: Index for power plants, $i \in \{1, 2, ..., 6\}$
- $t$: Index for time periods, $t \in \{1, 2, ..., 15\}$
- $S_i$: Startup cost for power plant $i$
- $D_i$: Shutdown cost for power plant $i$
- $F_i$: Fixed cost for power plant $i$
- $C_{it}$: Variable cost for power plant $i$ at time $t$
- $L_i$: Lower bound for output power of power plant $i$
- $U_i$: Upper bound for output power of power plant $i$
- $I_i$: Maximum power increment for power plant $i$
- $J_i$: Maximum power decrement for power plant $i$
- $D_t$: Power demand at time $t$

Decision Variables:
- $x_{it}$: Output power of plant $i$ at time $t$
- $y_{it}$: Binary variable indicating whether power plant $i$ is turned on at time $t$
- $z_{it}$: Binary variable indicating whether power plant $i$ is turned off at time $t$

Objective Function:
The objective is to minimize the total cost, which includes startup costs, shutdown costs, fixed costs, and variable costs:
\[
\min \sum_{i=1}^{6} \sum_{t=1}^{15} (S_i y_{it} + D_i z_{it} + F_i y_{it} + C_{it} x_{it})
\]

Constraints:
1. Power output bounds:
\[
L_i y_{it} \leq x_{it} \leq U_i y_{it} \quad \forall i \in \{1, 2, ..., 6\}, \forall t \in \{1, 2, ..., 15\}
\]

2. Power output change bounds:
\[
-J_i y_{i,t-1} \leq x_{it} - x_{i,t-1} \leq I_i y_{i,t-1} \quad \forall i \in \{1, 2, ..., 6\}, \forall t \in \{2, 3, ..., 15\}
\]

3. Power plant status change bounds:
\[
y_{it} - y_{i,t-1} \leq y_{it} \quad \forall i \in \{1, 2, ..., 6\}, \forall t \in \{2, 3, ..., 15\}
\]

4. Power output meets demand and security margin:
\[
\sum_{i=1}^{6} x_{it} \geq 1.1 D_t \quad \forall t \in \{1, 2, ..., 15\}
\]

5. Power plant status:
\[
y_{it} - z_{it} = y_{i,t-1} - z_{i,t-1} + y_{it} - y_{i,t-1} \quad \forall i \in \{1, 2, ..., 6\}, \forall t \in \{1, 2, ..., 15\}
\]

This optimization model is a mixed-integer linear programming (MILP) problem, which can be solved using various optimization techniques and software packages.

In [6]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- $i$: Index for power plants, $i \in \{1, 2, ..., 6\}$
- $t$: Index for time periods, $t \in \{1, 2, ..., 15\}$
- $S_i$: Startup cost for power plant $i$
- $D_i$: Shutdown cost for power plant $i$
- $F_i$: Fixed cost for power plant $i$
- $C_{it}$: Variable cost for power plant $i$ at time $t$
- $L_i$: Lower bound for output power of power plant $i$
- $U_i$: Upper bound for output power of power plant $i$
- $I_i$: Maximum power increment for power plant $i$
- $J_i$: Maximum power decrement for power plant $i$
- $D_t$: Power demand at time $t$

Decision Variables:
- $x_{it}$: Output power of plant $i$ at time $t$
- $y_{it}$: Binary variable indicating whether power plant $i$ is turned on at time $t$
- $z_{it}$: Binary variable indicating whether power plant $i$ is turned off at time $t$

Objective Function:
The 

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here is a Python code using Pyomo to model and solve the optimization problem:

```python
import pyomo.environ as pyo

# Sample data
data = {
    'I': range(1, 7),  # power plants
    'T': range(1, 16),  # time periods
    'S': {1: 100, 2: 120, 3: 150, 4: 80, 5: 110, 6: 130},  # startup costs
    'D': {1: 80, 2: 90, 3: 110, 4: 70, 5: 100, 6: 120},  # shutdown costs
    'F': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # fixed costs
    'C': {(i, t): 10 * i + t for i in range(1, 7) for t in range(1, 16)},  # variable costs
    'L': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # lower bounds
    'U': {1: 200, 2: 220, 3: 250, 4: 180, 5: 220, 6: 260},  # upper bounds
    'I': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # max increment
    'J': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # max decrement
    'D_t': {t: 100 * t for t in range(1, 16)}  # power demand
}

# Create the Pyomo model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=data['I'])
model.T = pyo.Set(initialize=data['T'])

# Parameters
model.S = pyo.Param(model.I, initialize=data['S'])
model.D = pyo.Param(model.I, initialize=data['D'])
model.F = pyo.Param(model.I, initialize=data['F'])
model.C = pyo.Param(model.I, model.T, initialize=data['C'])
model.L = pyo.Param(model.I, initialize=data['L'])
model.U = pyo.Param(model.I, initialize=data['U'])
model.I = pyo.Param(model.I, initialize=data['I'])
model.J = pyo.Param(model.I, initialize=data['J'])
model.D_t = pyo.Param(model.T, initialize=data['D_t'])

# Decision variables
model.x = pyo.Var(model.I, model.T, domain=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, model.T, domain=pyo.Binary)
model.z = pyo.Var(model.I, model.T, domain=pyo.Binary)

# Objective function
def obj_function(model):
    return sum(model.S[i] * model.y[i, t] + model.D[i] * model.z[i, t] + model.F[i] * model.y[i, t] + model.C[i, t] * model.x[i, t]
               for i in model.I for t in model.T)

model.obj = pyo.Objective(rule=obj_function, sense=pyo.minimize)

# Constraints
def power_output_bounds(model, i, t):
    return model.L[i] * model.y[i, t] <= model.x[i, t] <= model.U[i] * model.y[i, t]

model.power_output_bounds = pyo.Constraint(model.I, model.T, rule=power_output_bounds)

def power_output_change_bounds(model, i, t):
    if t == 1:
        return pyo.Constraint.Skip
    else:
        return -model.J[i] * model.y[i, t-1] <= model.x[i, t] - model.x[i, t-1] <= model.I[i] * model.y[i, t-1]

model.power_output_change_bounds = pyo.Constraint(model.I, model.T, rule=power_output_change_bounds)

def power_plant_status_change_bounds(model, i, t):
    if t == 1:
        return pyo.Constraint.Skip
    else:
        return model.y[i, t] - model.y[i, t-1] <= model.y[i, t]

model.power_plant_status_change_bounds = pyo.Constraint(model.I, model.T, rule=power_plant_status_change_bounds)

def power_output_meets_demand(model, t):
    return sum(model.x[i, t] for i in model.I) >= 1.1 * model.D_t[t]

model.power_output_meets_demand = pyo.Constraint(model.T, rule=power_output_meets_demand)

def power_plant_status(model, i, t):
    if t == 1:
        return model.y[i, t] - model.z[i, t] == model.y[i, t]
    else:
        return model.y[i, t] - model.z[i, t] == model.y[i, t-1] - model.z[i, t-1] + model.y[i, t] - model.y[i, t-1]

model.power_plant_status = pyo.Constraint(model.I, model.T, rule=power_plant_status)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(results)
```

This code creates a Pyomo model based on the given optimization problem and uses the GLPK solver to solve it. The results of the optimization are printed at the end.

In [10]:
print(response_2_text)

Here is a Python code using Pyomo to model and solve the optimization problem:

```python
import pyomo.environ as pyo

# Sample data
data = {
    'I': range(1, 7),  # power plants
    'T': range(1, 16),  # time periods
    'S': {1: 100, 2: 120, 3: 150, 4: 80, 5: 110, 6: 130},  # startup costs
    'D': {1: 80, 2: 90, 3: 110, 4: 70, 5: 100, 6: 120},  # shutdown costs
    'F': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # fixed costs
    'C': {(i, t): 10 * i + t for i in range(1, 7) for t in range(1, 16)},  # variable costs
    'L': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # lower bounds
    'U': {1: 200, 2: 220, 3: 250, 4: 180, 5: 220, 6: 260},  # upper bounds
    'I': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # max increment
    'J': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # max decrement
    'D_t': {t: 100 * t for t in range(1, 16)}  # power demand
}

# Create the Pyomo model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=data['I'])
model.T = pyo.Set(init

### Code Executability

In [11]:
import pyomo.environ as pyo

# Sample data
data = {
    'I': range(1, 7),  # power plants
    'T': range(1, 16),  # time periods
    'S': {1: 100, 2: 120, 3: 150, 4: 80, 5: 110, 6: 130},  # startup costs
    'D': {1: 80, 2: 90, 3: 110, 4: 70, 5: 100, 6: 120},  # shutdown costs
    'F': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # fixed costs
    'C': {(i, t): 10 * i + t for i in range(1, 7) for t in range(1, 16)},  # variable costs
    'L': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # lower bounds
    'U': {1: 200, 2: 220, 3: 250, 4: 180, 5: 220, 6: 260},  # upper bounds
    'I': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # max increment
    'J': {1: 50, 2: 60, 3: 70, 4: 40, 5: 60, 6: 80},  # max decrement
    'D_t': {t: 100 * t for t in range(1, 16)}  # power demand
}

# Create the Pyomo model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=data['I'])
model.T = pyo.Set(initialize=data['T'])

# Parameters
model.S = pyo.Param(model.I, initialize=data['S'])
model.D = pyo.Param(model.I, initialize=data['D'])
model.F = pyo.Param(model.I, initialize=data['F'])
model.C = pyo.Param(model.I, model.T, initialize=data['C'])
model.L = pyo.Param(model.I, initialize=data['L'])
model.U = pyo.Param(model.I, initialize=data['U'])
model.I = pyo.Param(model.I, initialize=data['I'])
model.J = pyo.Param(model.I, initialize=data['J'])
model.D_t = pyo.Param(model.T, initialize=data['D_t'])

# Decision variables
model.x = pyo.Var(model.I, model.T, domain=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, model.T, domain=pyo.Binary)
model.z = pyo.Var(model.I, model.T, domain=pyo.Binary)

# Objective function
def obj_function(model):
    return sum(model.S[i] * model.y[i, t] + model.D[i] * model.z[i, t] + model.F[i] * model.y[i, t] + model.C[i, t] * model.x[i, t]
               for i in model.I for t in model.T)

model.obj = pyo.Objective(rule=obj_function, sense=pyo.minimize)

# Constraints
def power_output_bounds(model, i, t):
    return model.L[i] * model.y[i, t] <= model.x[i, t] <= model.U[i] * model.y[i, t]

model.power_output_bounds = pyo.Constraint(model.I, model.T, rule=power_output_bounds)

def power_output_change_bounds(model, i, t):
    if t == 1:
        return pyo.Constraint.Skip
    else:
        return -model.J[i] * model.y[i, t-1] <= model.x[i, t] - model.x[i, t-1] <= model.I[i] * model.y[i, t-1]

model.power_output_change_bounds = pyo.Constraint(model.I, model.T, rule=power_output_change_bounds)

def power_plant_status_change_bounds(model, i, t):
    if t == 1:
        return pyo.Constraint.Skip
    else:
        return model.y[i, t] - model.y[i, t-1] <= model.y[i, t]

model.power_plant_status_change_bounds = pyo.Constraint(model.I, model.T, rule=power_plant_status_change_bounds)

def power_output_meets_demand(model, t):
    return sum(model.x[i, t] for i in model.I) >= 1.1 * model.D_t[t]

model.power_output_meets_demand = pyo.Constraint(model.T, rule=power_output_meets_demand)

def power_plant_status(model, i, t):
    if t == 1:
        return model.y[i, t] - model.z[i, t] == model.y[i, t]
    else:
        return model.y[i, t] - model.z[i, t] == model.y[i, t-1] - model.z[i, t-1] + model.y[i, t] - model.y[i, t-1]

model.power_plant_status = pyo.Constraint(model.I, model.T, rule=power_plant_status)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(results)

2024-05-22 21:09:09,533 ERROR pyomo.core: Constructing component 'I' from data=None failed:
KeyError: "Cannot treat the scalar component 'I' as an indexed component"


KeyError: "Cannot treat the scalar component 'I' as an indexed component"

### Solution Correctness